In [1]:
import os, sys
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

import pickle
import pandas as pd

#force reimport
if 'pygents.util' in sys.modules:
    del sys.modules['pygents.util']
if 'pygents.text' in sys.modules:
    del sys.modules['pygents.text']
if 'pygents.plot' in sys.modules:
    del sys.modules['pygents.plot']
if 'pygents.token' in sys.modules:
    del sys.modules['pygents.token']
if 'pygents.token_plot' in sys.modules:
    del sys.modules['pygents.token_plot']


from pygents.token import *
from pygents.text import tokenize_with_lexicon
from pygents.plot import plot_bars, plot_dict, matrix_plot
from pygents.token_plot import *

In [2]:
def compact_whitespaces(s):
    s = s.strip()
    s = s.replace('\n',' ')
    s = s.replace('\r',' ')
    s = s.replace('\t',' ')
    oldlen = len(s)
    while True: 
        s = s.replace('  ',' ')
        l = len(s) 
        if l == oldlen:
            break
        oldlen = l
    return s

assert compact_whitespaces("  a  b\tc\n\rd \te\t\tg  ") == 'a b c d e g'   
assert compact_whitespaces("   aa    aa.    bb bb! cc cc? dd dd.   ") == 'aa aa. bb bb! cc cc? dd dd.'


In [22]:
def sentencinize_with_patterns(patterns,text,cased=False,compact=True):
    # handle list of text
    if type(text) == list:
        chunks = []
        for chunk in text:
            chunks.extend(sentencinize_with_patterns(patterns,chunk,cased,compact))
        return chunks
    
    # handle single text - recursively with the every pattern
    alex = [pat+" " for pat in patterns]
    alex.sort(key=len,reverse=True) # append patterns with spaces sorted by len in reverse
    if compact:
        text = compact_whitespaces(text)
    if cased: #if need to spend time on lowercasing non-lowercased text
        text = text.lower()
    tokens = []
    start = 0
    cur = 0
    length = len(text)
    while cur < length:
        subtext = text[cur:]
        #print(subtext)
        for al in alex:
            matched = False
            if subtext.startswith(al):
                tokens.append(text[start:cur+len(al)-1])
                cur += len(al)
                start = cur
                matched = True
                break
        if not matched:
            cur += 1
    if start < cur:
        tokens.append(text[start:cur])
    return tokens

assert str(sentencinize_with_patterns([".","?","!"],"aa aa. bb bb! cc cc? dd dd")) == "['aa aa.', 'bb bb!', 'cc cc?', 'dd dd']"
assert str(sentencinize_with_patterns([".","?","!"],"aa aa. bb bb! cc cc? dd dd.")) == "['aa aa.', 'bb bb!', 'cc cc?', 'dd dd.']"
assert str(sentencinize_with_patterns([".","?","!"],"   aa    aa.    bb bb! cc cc? dd dd.   ")) == "['aa aa.', 'bb bb!', 'cc cc?', 'dd dd.']"


In [49]:
def load_text_lines(path,compact=True,lower=True,debug=False):
    with open(path,errors='ignore') as f:
        lines = []
        for l in f.readlines():
            if compact:
                l = compact_whitespaces(l)
            if lower:
                l = l.lower()
            lines.append(l)
        return lines


In [50]:
path = '../../nlp/corpora/Chinese/'
test_df = pd.read_csv(os.path.join(path,'magicdata/zh_en_ru_100/CORPUS_ZH_EN_RU.txt'),delimiter='\t')
test_texts = list(test_df['en'])
print(len(test_texts))
test_df[['en']]


100


,en
0,What about medical insurance? As for my family...
1,"For those who have insurance, according to the..."
2,Need to realize the importance of having insur...
3,"In fact, this phenomenon is indeed very common..."
4,It is really necessary for this generation of ...
...,...
95,Ant Insurance does not only offer car insuranc...
96,"However, when buying a house, except for the d..."
97,This kind of financial investment has certain ...
98,"If your investment orientation is right, then ..."


In [51]:
test_sents = sentencinize_with_patterns([".","?","!"],test_texts)
print(len(test_texts))
print(len(test_sents))


100
105


In [52]:
sentencinize_with_patterns([".","?","!"],test_texts[0:4])

['What about medical insurance?',
 'As for my family, either an adult or a child will buy insurance.',
 'For those who have insurance, according to the insurance contract, they will get a compensation of 300 thousand yuan.',
 'Need to realize the importance of having insurance.',
 'In fact, this phenomenon is indeed very common, for instance, for personal accident insurance, the more you buy, the more you insure.']

In [53]:
lines = load_text_lines('data/corpora/Brown/brown_nolines_en.txt')
print(len(lines))
for l in lines[0:100]:
    print(l)


14342
the fulton county grand jury said friday an investigation of atlanta's recent primary election produced "no evidence" that any irregularities took place.
the jury further said in term-end presentments that the city executive committee, which had over-all charge of the election, "deserves the praise and thanks of the city of atlanta" for the manner in which the election was conducted.
the september-october term jury had been charged by fulton superior court judge durwood pye to investigate reports of possible "irregularities" in the hard-fought primary which was won by mayor-nominate ivan allen jr..
"only a relative handful of such reports was received", the jury said, "considering the widespread interest in the election, the number of voters and the size of this city".
the jury said it did find that many of georgia's registration and election laws "are outmoded or inadequate and often ambiguous".
it recommended that fulton legislators act "to have these laws studied and revised t

In [54]:
sents = sentencinize_with_patterns([".","?","!"],lines)
print(len(sents))
for l in sents[0:100]:
    print(l)

57200
the fulton county grand jury said friday an investigation of atlanta's recent primary election produced "no evidence" that any irregularities took place.
the jury further said in term-end presentments that the city executive committee, which had over-all charge of the election, "deserves the praise and thanks of the city of atlanta" for the manner in which the election was conducted.
the september-october term jury had been charged by fulton superior court judge durwood pye to investigate reports of possible "irregularities" in the hard-fought primary which was won by mayor-nominate ivan allen jr..
"only a relative handful of such reports was received", the jury said, "considering the widespread interest in the election, the number of voters and the size of this city".
the jury said it did find that many of georgia's registration and election laws "are outmoded or inadequate and often ambiguous".
it recommended that fulton legislators act "to have these laws studied and revised t

In [41]:
tokenizer = DelimiterTokenizer()

In [61]:
text = " ".join(lines)
print(len(text))

5971746


In [60]:
toks = tokenizer.tokenize(text)
print(len(toks))
print(len(text)/len(toks))


2172559
2.748715224764897


In [57]:
text[:500]


'the fulton county grand jury said friday an investigation of atlanta\'s recent primary election produced "no evidence" that any irregularities took place. the jury further said in term-end presentments that the city executive committee, which had over-all charge of the election, "deserves the praise and thanks of the city of atlanta" for the manner in which the election was conducted. the september-october term jury had been charged by fulton superior court judge durwood pye to investigate report'

In [59]:
print(tokenizer.tokenize(text[:500]))


['the', ' ', 'fulton', ' ', 'county', ' ', 'grand', ' ', 'jury', ' ', 'said', ' ', 'friday', ' ', 'an', ' ', 'investigation', ' ', 'of', ' ', "atlanta's", ' ', 'recent', ' ', 'primary', ' ', 'election', ' ', 'produced', ' ', '"', 'no', ' ', 'evidence', '"', ' ', 'that', ' ', 'any', ' ', 'irregularities', ' ', 'took', ' ', 'place', '.', ' ', 'the', ' ', 'jury', ' ', 'further', ' ', 'said', ' ', 'in', ' ', 'term-end', ' ', 'presentments', ' ', 'that', ' ', 'the', ' ', 'city', ' ', 'executive', ' ', 'committee', ',', ' ', 'which', ' ', 'had', ' ', 'over-all', ' ', 'charge', ' ', 'of', ' ', 'the', ' ', 'election', ',', ' ', '"', 'deserves', ' ', 'the', ' ', 'praise', ' ', 'and', ' ', 'thanks', ' ', 'of', ' ', 'the', ' ', 'city', ' ', 'of', ' ', 'atlanta', '"', ' ', 'for', ' ', 'the', ' ', 'manner', ' ', 'in', ' ', 'which', ' ', 'the', ' ', 'election', ' ', 'was', ' ', 'conducted', '.', ' ', 'the', ' ', 'september-october', ' ', 'term', ' ', 'jury', ' ', 'had', ' ', 'been', ' ', 'charged', 

In [34]:
#https://towardsdatascience.com/5-simple-ways-to-tokenize-text-in-python-92c6804edfc4
#https://www.oreilly.com/content/how-can-i-tokenize-a-sentence-with-python/
import nltk

#TODO run this to make it working
nltk.download('punkt')
SENT_DETECTOR = nltk.data.load('tokenizers/punkt/english.pickle')

#def text_to_sentences(text):
#    return nltk.sent_tokenize(text)


[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>


In [8]:
#TODO:
 
# Exceptions on false breaking on ". " on Brown!?
# Sentencinoze Brown with NLTK!?

